# Quantitative Methods

In [1]:
import pandas as pd
import numpy as np

## Geometric Mean

In [2]:
from scipy.stats.mstats import gmean 

In [3]:
# The most recent five years of total returns for two US large-cap value equity mutual funds in 2018
d = {'Year': [2013, 2014, 2015, 2016, 2017], 
     'Selected American Shares (SLASX) %': [34.90, 6.13, 2.69, 11.66, 21.77], 
     'T. Rowe Price Equity Income (PRFDX)%':[31.69, 7.75, -7.56, 18.25, 16.18]}
df = pd.DataFrame(data=d)
df

,Year,Selected American Shares (SLASX) %,T. Rowe Price Equity Income (PRFDX)%
0,2013,34.90,31.69
1,2014,6.13,7.75
2,2015,2.69,-7.56
3,2016,11.66,18.25
4,2017,21.77,16.18


In [4]:
# Calculate the geometric mean return of SLASX
slasx = df['Selected American Shares (SLASX) %'].tolist()

\begin{equation*}
R_{t} + 1 = \sqrt[t]{(1+R_{1})(1+R_{2})...(1+R_{t})}
\end{equation*}


In [5]:
# Redefine returns to make them positive by adding 1.0
g_slasx = list(map(lambda i: i/100+1, slasx))

In [6]:
gmean(g_slasx) -1

0.14858526227425917

\begin{equation*}
G = \sqrt[n]{x_{1}x_{2}...x_{n}}
\end{equation*}with $X_{i}≥0$ for i=1,2,…,n.


In [7]:
# Calculate the arithmetic mean return of SLASX and contrast it to the fund’s geometric mean return
df['Selected American Shares (SLASX) %'].mean()

15.429999999999998

In [8]:
# Calculate the geometric mean return of PRFDX
g_prfdx = list(map(lambda i: i/100+1, df['T. Rowe Price Equity Income (PRFDX)%'].tolist()))
gmean(g_prfdx) - 1

0.1249999599109004

In [9]:
# Calculate the arithmetic mean return of PRFDX and contrast it to the fund’s geometric mean return
df['T. Rowe Price Equity Income (PRFDX)%'].mean()

13.262

## Harmonic Mean and Weighted Mean
<br>
Use case 1: "Cost averaging" which requires periodic investment of a fixed amount of money. 

In [10]:
import statistics

In [11]:
price_per_share = [12.1, 13.5, 17, 11.98, 15]

In [12]:
print("Harmonic Mean is % s " % (statistics.harmonic_mean(price_per_share))) 

Harmonic Mean is 13.67310629317005 


## Quantiles

**Real time data application**

In [13]:
# Web scraping Euro Stoxx 50 data
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://www.dividendmax.com/market-index-constituents/euro-stoxx-50")
soup = BeautifulSoup(html, "html.parser")
table = soup.findAll("table")[0]
rows = table.findAll("tr")

with open("editors.csv", "wt+", newline="") as f:
    writer = csv.writer(f)
    for row in rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)

In [14]:
euro_stoxx = pd.read_csv('editors.csv')

In [25]:
euro_stoxx.head()

,Company,Exchange,Price,Currency,\nMarket Cap\n\n
0,🇫🇷AIAir Liquide S.A,Euronext Paris (France),135.55,EUR,€42.3bn
1,🇩🇪ALVAllianz SE,Frankfurt Stock Exchange (Germany),164.38,EUR,€74.2bn
2,🇧🇪ABIAnheuser-Busch In Bev SA/NV,Euronext Brussels (Belgium),44.78,EUR,€71.9bn
3,🇱🇺MTLArcelorMittal,Luxembourg Stock Exchange (Luxembourg),12.94,EUR,€19.5bn
4,🇳🇱ASMLASML Holding NV,Euronext Amsterdam (Netherlands),308.15,EUR,€129.4bn


Quantiles in practice: determing or approximating the value of a percentile (when the location is not a whole number). 

\begin{equation*}
L_{y} = (n+1) * \frac{y}{100}
\end{equation*}

In [124]:
# Make a function for calculating Ly with a given column and desired percentile
def loc_y(column, percentile):
    n = len(column)
    loc = (n+1)*percentile/100
    return loc

#### Situation I
When the location, $L_{y}$, is a whole number, the location corresponds to an actual observation. 
<br>$X_{i}$ is defined as the value of the observation in the ith ($i = L_{60}$) position of the data sorted in ascending order (i.e., $P_{30} =8.16$).

#### Situation II
When $L_{y}$ is not a whole number or integer, $L_{y}$ lies between the two closest integer numbers (one above and one below), and we use **linear interpolation** between those two places to determine $P_{y}$.

Making a function to calculate $P_{y}$ given an array (observation) and the percentile of interest. 

**Practice based on CFA textbook example data:**

In [36]:
his_euro_stoxx = pd.read_csv('euro_stoxx_2018_nov.csv', delimiter=';')

In [38]:
his_euro_stoxx.head()

,No.,Company,Market Cap (Euro Billion)
0,1,RWE,9.9
1,2,Carrefour,12.2
2,3,E.ON,15.5
3,4,Inditex,15.5
4,5,Unibail Rodamco,16.1


In [193]:
# 1. Turn the column into a list and sort ascending
# 2. Get location
# 3. Calculate Py based on the location: if y is integer, direct indexing; if not an integer, use linear interpolation
def p_given_percent(column, percentile):
    list1 = column.tolist()
    sorted_list = sorted(list1)
    n = len(sorted_list)
    loc = (n+1)*percentile/100
    if loc.is_integer():
        print('Ly is an integer,', percentile, '% of data lies at or below Py =', sorted_list[(int(loc)-1)])
    else:
        i = int(loc)
        P_approx = float(sorted_list[i-1]) + (float(loc) - i) * (float(sorted_list[i]) - float(sorted_list[i-1]))
        print('Ly is not an integer, Py is approximately', P_approx, sorted_list[i-1], 'and', sorted_list[i], 'bracket Py from below and above, respectively.')
        
# WIP        

In [147]:
?isinstance

In [194]:
# 10th percentile     
p_given_percent(his_euro_stoxx['Market Cap (Euro Billion)'], 10)

Ly is not an integer, Py is approximately 16.16 16.1 and 16.7 bracket Py from below and above, respectively.


In [195]:
# 90th percentile     
p_given_percent(his_euro_stoxx['Market Cap (Euro Billion)'], 90)

Ly is not an integer, Py is approximately 104.7 104.7 and 104.7 bracket Py from below and above, respectively.


In [196]:
# Second quartile
p_given_percent(his_euro_stoxx['Market Cap (Euro Billion)'], 50)

Ly is not an integer, Py is approximately 37.5 36.6 and 38.4 bracket Py from below and above, respectively.


In [197]:
# First quintile 
p_given_percent(his_euro_stoxx['Market Cap (Euro Billion)'], 20)

Ly is not an integer, Py is approximately 20.52 20.1 and 22.2 bracket Py from below and above, respectively.


In [199]:
# Value of the median
np.median(his_euro_stoxx['Market Cap (Euro Billion)'])

37.5

In [ ]:
# How many quintiles are there, and to what percentiles do the quintiles correspond? 
# Answer: 4, P20, P40, P60 and P80

Real time Euro Stoxx data application. Example 1 with $L_{y}$ as a whole number. 

**Solution 1.** Step by step

In [172]:
# Change Price data type as float, get rid of € symbol
euro_stoxx['Price'] = euro_stoxx['Price'].str.replace('€', '')
price = pd.to_numeric(euro_stoxx['Price']).tolist()
price = sorted(price)

In [182]:
# Obtaining the location (i) at 60th percentile
loc_y(euro_stoxx['Company'], 60)

30.0

Since $L_{y}$ is a whole number, $P_{60}$ = $X_{30}$

In [167]:
# Get value of X30 (= P60)
price[29]

44.49

**Solution 2.** Using `p_given_stoxx()` funtion

In [198]:
# Using p_given_percent() funtion:
p_given_percent(euro_stoxx['Price'], 60)

Ly is an integer, 60 % of data lies at or below Py = 35.08


60% of the values in this data set lies at or below €44.78 in price.  

**Solution 3.** Using Numpy's `percentile` function

To verify the above calculation for future efficiency, I proceed with using Numpy's `percentile` function to obtain the value. 

In [169]:
# Obtain the value at 60th percentile
np.percentile(price, 60)
# But this is not in the observation, what went wrong?

43.27399999999999

Example 2 with $L_{y}$ as a fraction.  

**Solution 1.** Step by step

In [204]:
# Obtaining the location (i) at 75th percentile
print('The location is at', loc_y(euro_stoxx['Company'], 75))

The location is at 37.5


The value of $P_{y}$ need to be approximated between $X_{37}$ and $X_{38}$.

In [207]:
# Obtain values on both sides of Ly
X37 = price[36]
X38 = price[37]

In [208]:
# Linear interpolation
P75_approx = X37 + (37.5 - 37) * (X38 - X37)
P75_approx

79.1

**Solution 2.** Using `p_given_percent()` function

In [176]:
p_given_percent(euro_stoxx['Price'], 75)

Ly is not an integer, Py is 28.26


**Solution 3.** Verify with Numpy `percentile`

In [173]:
# Obtain the value at 75th percentile
np.percentile(price, 75)

71.06

## Quantiles in Investment Practice
<br>
Quantiles are used in portfolio performance evaluation as well as in investment strategy development and research.

**User case 1. Ranking performance—for example, the performance of portfolios.**<br>


**User case 2. Investment research<br>**
Analysts refer to a group defined by a particular quantile as that quantile. For example, analysts often refer to the set of companies with returns falling below the 10th percentile cutoff point as the bottom return decile. Dividing data into quantiles based on some characteristic allows analysts to evaluate the impact of that characteristic on a quantity of interest. For instance, empirical finance studies commonly rank companies based on the market value of their equity and then sort them into deciles. The 1st decile contains the portfolio of those companies with the smallest market values, and the 10th decile contains those companies with the largest market value. Ranking companies by decile allows analysts to compare the performance of small companies with large ones.

**Investment style based on liquidity**<br>
Ibbotson et al. (2018): Investment style based on liquidity—buying stocks of less liquid stocks and selling stocks of more liquid stocks.<br>
Computed:
- yearly measures of liquidity as the annual share turnover (the sum of the 12 monthly volumes divided by each month’s shares outstanding);
- size as the year-end market capitalization; 
- value as the trailing earnings-to-price ratio as of the year end;
- momentum as the annual return. 

Step 1. assigned one-fourth of the total sample with the lowest liquidity in a year to Quartile 1 and the one-fourth with the highest liquidity in that year to Quartile 4.

Step 2. treating each quartile group as a portfolio composed of equally weighted stocks, they measured the returns on each liquidity quartile in the following year (so that the quartiles are constructed “before the fact”.) 

## Measure of Dispersion
The most common measures of *(absolute) dispersion*: range, mean absolute deviation, variance, and standard deviation.

#### Mean Absolute Deviation (MAD)

In [ ]:
pd.to

In [218]:
euro_stoxx['Price'].mad()

nan

In [219]:
pd.Series(price).mad()

59.49108704706372